In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [2]:
raw_data = pd.read_csv("./product_regression.csv", index_col = 0)
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1551 entries, 0 to 1550
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   회사코드      1551 non-null   int64  
 1   사업장코드     1551 non-null   int64  
 2   생산일자      1551 non-null   object 
 3   생산순번      1551 non-null   int64  
 4   생산지시생성일자  1551 non-null   object 
 5   생산지시순번    1551 non-null   int64  
 6   생산진행순번    57 non-null     float64
 7   품목코드      1551 non-null   object 
 8   target    1551 non-null   float64
 9   Line코드    1551 non-null   object 
 10  실적작업장     1551 non-null   object 
 11  표준ST      1551 non-null   float64
 12  작업상태      1551 non-null   int64  
 13  작업시작일시    1551 non-null   object 
 14  종료예상일시    0 non-null      float64
 15  작업종료일시    1551 non-null   object 
 16  투입인원      1551 non-null   int64  
 17  작업시간      1551 non-null   float64
 18  등록자ID     1551 non-null   int64  
 19  등록시간      1551 non-null   object 
dtypes: float64(5), int64(7), objec

In [3]:
df = raw_data.copy()
df.tail()

,회사코드,사업장코드,생산일자,생산순번,생산지시생성일자,생산지시순번,생산진행순번,품목코드,target,Line코드,실적작업장,표준ST,작업상태,작업시작일시,종료예상일시,작업종료일시,투입인원,작업시간,등록자ID,등록시간
1546,139,0,2021-04-21 00:00:00,41,2021-04-15 00:00:00,58,NaN,72312724118,88.0,L21,A22,0.836,3,2021-04-21 13:50:54.000,NaN,2021-04-21 15:25:09.383,1,95.0,117,2021-04-21 13:50:52.523
1547,139,0,2021-04-21 00:00:00,42,2021-04-02 00:00:00,31,NaN,38120028,74.0,L21,A21,6.930,3,2021-04-20 15:00:09.000,NaN,2021-04-21 15:38:31.933,1,1478.0,120,2021-04-20 15:00:07.090
1548,139,0,2021-04-21 00:00:00,43,2021-04-13 00:00:00,11,NaN,38020117,150.0,L21,A21,1.452,3,2021-04-21 14:21:47.000,NaN,2021-04-21 15:41:20.297,1,80.0,117,2021-04-21 14:21:37.520
1549,139,0,2021-04-21 00:00:00,44,2021-04-02 00:00:00,31,NaN,38120028,6.0,L21,A21,6.930,3,2021-04-21 15:40:32.000,NaN,2021-04-21 15:41:52.460,1,1.0,117,2021-04-21 15:38:31.937
1550,139,0,2021-04-21 00:00:00,48,2021-04-13 00:00:00,15,NaN,38040050,179.0,L21,A22,2.343,3,2021-04-21 11:58:41.000,NaN,2021-04-21 16:26:10.020,1,268.0,117,2021-04-21 11:58:31.517


In [4]:
# 데이터 unique 값 확인
#for column in df.columns:
#    print(column,": ", df[column].unique())
#    print("")

In [5]:
# 전처리
# 등록자ID, 등록시간 : 생산실적 예측에 영향을 미치지 않을 컬럼이므로 제거
# Line코드 : L21 작업라인에 대한 데이터만 불러왔으므로 제거
# 회사코드, 사업장코드, 작업상태 : 모두 같은 값이므로 제거
# 생산진행순번 : 결측치 다수 이므로 제거
# 종료예상일시 : 값이 모두 결측치
# 작업시작일시 - 작업종료일시 = 작업시간 : 작업시작일시, 작업종료일시 제거
# 표준 ST(Standard Time) : 작업시간 기준에 대한 명시이므로 제거
# 작업시간 0인 row 제거
# 생산일자 = 작업종료일시 일자 : 제거
# 품목코드 : Regression 모델 적용을 위한 Encoding이 필요하나 생략
df_drop_columns = df.drop(['등록자ID','등록시간','Line코드','회사코드','사업장코드',
                           '작업상태','생산진행순번','종료예상일시','작업시작일시',
                           '작업종료일시','표준ST','생산일자','생산지시생성일자',
                           '품목코드'],axis=1)
df_drop_columns.head()

,생산순번,생산지시순번,target,실적작업장,투입인원,작업시간
0,9,55,365.0,A11,3,8.0
1,11,68,20.0,A23,2,3.0
2,14,59,3.0,A23,2,0.0
3,15,54,76.0,A23,2,0.0
4,17,44,300.0,A22,1,1.0


In [6]:
# feature engineering
# 작업시작일시 - 생산지시일지 : 생산대기시간
df["작업시작일시"] = pd.to_datetime(df["작업시작일시"])
df["생산지시생성일자"] = pd.to_datetime(df["생산지시생성일자"])
df_drop_columns['생산대기일수'] = df["작업시작일시"].dt.date - df["생산지시생성일자"].dt.date
df_added = df_drop_columns.copy()
del df_drop_columns
df_added.head()

,생산순번,생산지시순번,target,실적작업장,투입인원,작업시간,생산대기일수
0,9,55,365.0,A11,3,8.0,2 days
1,11,68,20.0,A23,2,3.0,2 days
2,14,59,3.0,A23,2,0.0,2 days
3,15,54,76.0,A23,2,0.0,2 days
4,17,44,300.0,A22,1,1.0,2 days


In [7]:
df_added.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1551 entries, 0 to 1550
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype          
---  ------  --------------  -----          
 0   생산순번    1551 non-null   int64          
 1   생산지시순번  1551 non-null   int64          
 2   target  1551 non-null   float64        
 3   실적작업장   1551 non-null   object         
 4   투입인원    1551 non-null   int64          
 5   작업시간    1551 non-null   float64        
 6   생산대기일수  1551 non-null   timedelta64[ns]
dtypes: float64(2), int64(3), object(1), timedelta64[ns](1)
memory usage: 96.9+ KB


In [8]:
df_added.head()

,생산순번,생산지시순번,target,실적작업장,투입인원,작업시간,생산대기일수
0,9,55,365.0,A11,3,8.0,2 days
1,11,68,20.0,A23,2,3.0,2 days
2,14,59,3.0,A23,2,0.0,2 days
3,15,54,76.0,A23,2,0.0,2 days
4,17,44,300.0,A22,1,1.0,2 days


In [9]:
df_added['실적작업장'].unique()

array(['A11', 'A23', 'A22', 'A21', 'A24', 'A12'], dtype=object)

In [10]:
# 실적작업장 One-Hot Encoding
df_encoded = pd.get_dummies(df_added, columns=['실적작업장'])
df_encoded.head()

,생산순번,생산지시순번,target,투입인원,작업시간,생산대기일수,실적작업장_A11,실적작업장_A12,실적작업장_A21,실적작업장_A22,실적작업장_A23,실적작업장_A24
0,9,55,365.0,3,8.0,2 days,1,0,0,0,0,0
1,11,68,20.0,2,3.0,2 days,0,0,0,0,1,0
2,14,59,3.0,2,0.0,2 days,0,0,0,0,1,0
3,15,54,76.0,2,0.0,2 days,0,0,0,0,1,0
4,17,44,300.0,1,1.0,2 days,0,0,0,1,0,0


In [11]:
feature_columns = ['생산순번','생산지시순번','투입인원','작업시간','생산대기일수',
                   '실적작업장_A11','실적작업장_A12','실적작업장_A21','실적작업장_A22','실적작업장_A23','실적작업장_A24']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df_encoded[feature_columns], df_encoded['target'], test_size=0.2, random_state=2021)

In [13]:
# Scaling
std_scaler = preprocessing.StandardScaler()

# Try using .loc[row_indexer,col_indexer] = value instead
for feature in feature_columns:
    std_scaler.fit(X_train[[feature]])
    X_train_scaled = std_scaler.transform(X_train[[feature]])
    X_train[[feature]] = X_train_scaled
    
    X_test_scaled = std_scaler.transform(X_test[[feature]])
    X_test[[feature]] = X_test_scaled

### 선형회귀

In [14]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
pred_lr = lr_model.predict(X_test)
mean_squared_error(y_test, pred_lr)

10792.233149079137

In [15]:
ridge_model = Ridge(alpha=10.0, random_state=2021) # alpha : Regularization 정도
ridge_model.fit(X_train, y_train)
pred_ridge = ridge_model.predict(X_test)
mean_squared_error(y_test, pred_ridge)

10785.036127403022

In [16]:
lasso_model = Lasso(alpha=10.0, random_state=2021)
lasso_model.fit(X_train, y_train)
pred_lasso = lasso_model.predict(X_test)
mean_squared_error(y_test, pred_lasso)

10755.313534490779

In [31]:
enet_model = ElasticNet(alpha=1, l1_ratio=0.8)
enet_model.fit(X_train, y_train)
pred_enet = enet_model.predict(X_test)
mean_squared_error(y_test, pred_enet)

10666.565078236972

### 다항회귀

In [57]:
poly_model = make_pipeline(preprocessing.PolynomialFeatures(2), Ridge())
#poly_model = make_pipeline(preprocessing.PolynomialFeatures(2), Lasso())
poly_model.fit(X_train, y_train)
pred_poly_model = poly_model.predict(X_test)
mean_squared_error(y_test, pred_poly_model)

8984.849506484363

### SVR

In [38]:
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
svr_rbf.fit(X_train, y_train)
pred_svr_rbf = svr_rbf.predict(X_test)
mean_squared_error(y_test, pred_svr_rbf)

9736.08241756469

In [42]:
svr_lin = SVR(kernel='linear', C=100, gamma='auto')
svr_lin.fit(X_train, y_train)
pred_svr_lin = svr_lin.predict(X_test)
mean_squared_error(y_test, pred_svr_lin)

12480.131838140362

In [47]:
svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=2, epsilon=0.1, coef0=1)
svr_poly.fit(X_train, y_train)
pred_svr_poly = svr_poly.predict(X_test)
mean_squared_error(y_test, pred_svr_poly)

9814.312655196461

## 보완점
- 전처리 과정에서 많은 컬럼들이 제거되어 생산실적 예측에 필요한 feature들 개수가 현저히 적어짐
    - 다른 테이블과의 join을 통해 생산실적 예측 학습에 쓰일만한 feature에 대한 추가적인 분석이 필요
- 품목코드의 경우 One-Hot Encoding을 하기엔 수가 많고 Label Encoding을 하기엔 회귀 모델에 영향을 줄 것 같아 제거
    - 품목별로 충분한 양의 row개수가 확보된다면 전체 라인에서 생산하는 실적량 예측이 아닌 특정 품목에 대한 예측을 수행하는 것이 오차를 줄일 수 있는 방법이라 생각됨

In [18]:
# 품목별 row 개수
for code in df['품목코드'].unique():
    print(code, ": ", len(df.query('품목코드=="{0}"'.format(code))))

74372124001 :  46
72313460304 :  52
72311176262 :  10
72311176261 :  52
38020088 :  16
72313460305 :  13
72313460324 :  14
72313460327 :  17
72313460328 :  2
72313471306 :  1
38060134 :  17
72312724118 :  46
38020082 :  9
99313460672 :  12
72312724117 :  7
72312724126 :  8
38060132 :  13
72311176264 :  16
72313466301 :  35
73263466359 :  20
73263466360 :  5
73263466356 :  1
38080008 :  4
38080010 :  13
38080012 :  5
38060172 :  2
38060173 :  3
38060174 :  2
3806013200 :  1
38060133 :  17
38020038 :  4
38020039 :  6
38020084 :  10
38020123 :  17
37060012 :  7
37060011 :  7
72313460348 :  7
72313460302 :  21
72312724127 :  4
74372124002 :  3
38040051 :  1
38020118 :  21
72311176301 :  20
38020117 :  21
38040050 :  13
72313460301 :  48
37020023 :  2
3808001000 :  15
38060054 :  11
38060055 :  9
73261176006 :  6
73263466363 :  19
38040028 :  7
37020022 :  8
37020024 :  3
37020030 :  3
37020032 :  3
38040019 :  1
38040003 :  3
38040020 :  9
38040027 :  1
38040023 :  1
38040024 :  6
38040007